In [1]:
import sys
import os

sys.path.append("../Pre_processing")

from Data_Preparation_Library import *
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

image_size=128
maxBatchId = 24


Using TensorFlow backend.


In [2]:
#selectedBatches=["6","7","8","9","10","11","12","13","14","15","16","17","18","19","20"]
selectedBatches=[str(i) for i in range(maxBatchId)]

extra_folder=str(image_size)+"_"+str(image_size)
batch_data_object = []
for i in selectedBatches:
    with open(os.path.join(temp_path,extra_folder,'full_data_object_' + i + '.p'), 'rb') as handle:
        batch_data_object+=pickle.load(handle)

data_train = batch_data_object[0:int(len(batch_data_object)*2/3)]


In [3]:
batch_data_test_object = []
for i in range(8):
    with open(os.path.join(temp_path,extra_folder,'test_data_object_' + str(i) + '.p'), 'rb') as handle:
        batch_data_test_object+=pickle.load(handle)
        
data_test  = batch_data_test_object

In [4]:
def flux(d):
  # TODO: get flux_i from model
    #if d.i_image != None:
    #    flux_i = sum(sum(d.i_image))
    #else:
    #    flux_i = 0
    flux_g = sum(sum(d.g_image_resized))
    #return flux_i+flux_g, flux_g-flux_i
    return flux_g

In [5]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers.core import Dropout

In [6]:
len(batch_data_test_object),batch_data_test_object[0].g_image_resized_reshaped.shape

(8447, (128, 128, 1))

In [7]:
version="v1"
res_folder="results_"+str(image_size)+"_on_"+str(image_size)
with open(os.path.join(os.path.join(output_path,res_folder),"encoder_results_train_"+version+"__all"),'rb') as handle:
    encoded_imgs1=pickle.load(handle)
    encoded_imgs_reshaped_train = [e.reshape(-1,1) for e in encoded_imgs1]

model_encoder=load_model(os.path.join(os.path.join(output_path,res_folder),"encoder_model_"+version+"__all"))

def get_g_Data_for_Autoencoder(data_object):
    X_list=[(data_object[index].g_image_resized_reshaped)/data_object[index].g_image_resized_reshaped.max() for index in range(len(data_object))]
    return np.asarray(X_list),np.asarray(X_list)

X_test_auto_encoder,y_test_auto_encoder=get_g_Data_for_Autoencoder(batch_data_test_object)

encoded_imgs2=model_encoder.predict(X_test_auto_encoder)
encoded_imgs_reshaped_test = [e.reshape(-1,1) for e in encoded_imgs2]


In [8]:
from skimage.measure import label
from scipy.ndimage.morphology import distance_transform_edt
from sklearn.decomposition import PCA
def clean_stars(img):
    mask = img >= np.percentile(img, 96)
    lbl = label(mask, background=0)
    center_label = lbl[int(image_size/2),int(image_size/2)]
    galaxy = lbl == center_label
    stars = (lbl != center_label) & (lbl != 0) 
    image_clean = img / (1+ 5* stars + distance_transform_edt(~galaxy))
    return mask, galaxy, lbl, image_clean
def get_features(img):
    f = {}
    __, galaxy, lbl, image_clean = clean_stars(img)
    f['flux'] = img[galaxy].sum()
    gxy = np.array(np.nonzero(galaxy)).T
    gxy_centered = gxy - gxy.mean(axis=0, keepdims=True)
    pca = PCA().fit(gxy_centered)
    f['pca_eigen1'] = pca.explained_variance_[0]
    try:
        f['pca_eigen2'] = pca.explained_variance_[1]
    except:
        f['pca_eigen2'] = 5.0
    f['ellipticity'] = f['pca_eigen1'] / f['pca_eigen2']
    f['minx'] = gxy[:,0].min()
    f['maxx'] = gxy[:,0].max()
    f['miny'] = gxy[:,1].min()
    f['maxy'] = gxy[:,1].max()
    distances = np.linalg.norm(gxy_centered, axis=1)
    f['max_dist'] = distances.max()
    f['median_dist'] = np.median(distances)
    f['2d_first_moment'] = (distances * distances * img[np.nonzero(galaxy)]).sum()
    f['max_lum'] = img[galaxy].max()
    f['center_lum'] = img[int(image_size/2),int(image_size/2)]
    return f

In [9]:
# accessible in the class: size of image, maximum of image, normalization factor of g + fitted value of normalization factor for i
# define a function to get the training and testing dataset

features_no_star = ['flux','pca_eigen1','pca_eigen2', 'ellipticity', 'max_dist', 'median_dist', '2d_first_moment',
                   'max_lum', 'center_lum']

def obtain_train_data():
    train_features = []
    train_labels = []
    for i in range(len(data_train)):
        if data_train[i].logMstar!=-99:
            e = [] #encoded_imgs_reshaped_train[i]
            d = data_train[i]
            #f1,f2 = flux(d)
            f = flux(d)
            #size_1=d.g_image.shape[0]
            #size_2=d.g_image.shape[1]
            img_max=d.g_image_resized.max()
            dist=d.Distance
            no_star_f = get_features(d.g_image_resized)
            train_features.append(np.append(e,
                np.array([f,
                #size_1,size_2,
                img_max,dist] + [no_star_f[feat_name] for feat_name in features_no_star])))
            #train_features.append(np.append(e,np.array([f])))
            train_labels.append(d.logMstar)
    return train_features, train_labels

train_features = np.asarray(obtain_train_data()[0])
train_labels = np.asarray(obtain_train_data()[1])

print(train_features.shape, train_labels.shape)

(15769, 12) (15769,)


In [10]:
def obtain_test_data():
    test_features = []
    test_labels = []
    for j in range(len(data_test)):
        if data_test[j].logMstar!=-99:
            e = [] # encoded_imgs_reshaped_test[j]
            d = data_test[j]

            #f1,f2 = flux(d)
            f = flux(d)
            #size_1=d.g_image.shape[0]
            #size_2=d.g_image.shape[1]
            img_max=d.g_image_resized.max()
            dist=d.Distance
            no_star_f = get_features(d.g_image_resized)
            test_features.append(np.append(e,
                np.array([f,
                                                       #size_1,size_2,
                   img_max,dist] + [no_star_f[feat_name] for feat_name in features_no_star])))
            test_labels.append(d.logMstar)

    return test_features, test_labels

test_features = np.asarray(obtain_test_data()[0])
test_labels = np.asarray(obtain_test_data()[1])

print(test_features.shape, test_labels.shape)

(8447, 12) (8447,)


In [11]:
train_features = np.nan_to_num(train_features)
test_features = np.nan_to_num(test_features)

In [12]:
# # ## transform features
# # train_features = StandardScaler().fit_transform(train_features)
# # test_features = StandardScaler().transform(test_features)
# # #features = MinMaxScaler().fit_transform(features)

X_scaler = StandardScaler()
train_features = X_scaler.fit_transform(train_features)
test_features = X_scaler.transform(test_features)


In [13]:
train_labels=np.exp(train_labels)

In [14]:
# train_features=train_features/(train_features.max(0).T)
# test_features=test_features/(train_features.max(0).T)

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.core import Dropout

In [16]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=train_features.shape[1]))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation="relu"))
model.compile(optimizer='Adadelta',
             loss='mse',
             metrics=['mse'])

In [17]:
model.fit(train_features, train_labels, epochs=50, batch_size=32,shuffle=True,
                validation_data=(test_features, test_labels))

Train on 15769 samples, validate on 8447 samples
Epoch 1/50
15769/15769 [==============================] - 2s - loss: 805194261.3381 - mean_squared_error: 805194261.3381 - val_loss: 2755064113.7332 - val_mean_squared_error: 2755064113.7332
Epoch 2/50
15769/15769 [==============================] - 1s - loss: 238296319.9980 - mean_squared_error: 238296319.9980 - val_loss: 2402042267.9275 - val_mean_squared_error: 2402042267.9275
Epoch 3/50
15769/15769 [==============================] - 1s - loss: 239258438.0514 - mean_squared_error: 239258438.0514 - val_loss: 2444561475.0382 - val_mean_squared_error: 2444561475.0382
Epoch 4/50
15769/15769 [==============================] - 1s - loss: 232530774.7900 - mean_squared_error: 232530774.7900 - val_loss: 2471426872.3400 - val_mean_squared_error: 2471426872.3400
Epoch 5/50
15769/15769 [==============================] - 1s - loss: 238706461.0961 - mean_squared_error: 238706461.0961 - val_loss: 2465733629.6967 - val_mean_squared_error: 2465733629.6

15769/15769 [==============================] - 1s - loss: 218017847.4303 - mean_squared_error: 218017847.4303 - val_loss: 2468527264.6251 - val_mean_squared_error: 2468527264.6251
Epoch 44/50
15769/15769 [==============================] - 1s - loss: 220201640.2937 - mean_squared_error: 220201640.2937 - val_loss: 2486914516.7828 - val_mean_squared_error: 2486914516.7828
Epoch 45/50
15769/15769 [==============================] - 1s - loss: 216533214.1716 - mean_squared_error: 216533214.1716 - val_loss: 2478502581.9003 - val_mean_squared_error: 2478502581.9003
Epoch 46/50
15769/15769 [==============================] - 1s - loss: 217858005.6957 - mean_squared_error: 217858005.6957 - val_loss: 2470076621.2364 - val_mean_squared_error: 2470076621.2364
Epoch 47/50
15769/15769 [==============================] - 1s - loss: 218079911.9488 - mean_squared_error: 218079911.9488 - val_loss: 2471030465.9017 - val_mean_squared_error: 2471030465.9017
Epoch 48/50
15769/15769 [===========================

In [18]:
from sklearn.metrics import mean_squared_error

test_predict = []
test_data_sdss_id=[]
for i in range(len(batch_data_test_object)):

    test_predict.append(model.predict(test_features[i].reshape(1,-1))[0][0])
    test_data_sdss_id.append(batch_data_test_object[i].SDSS_ID)
    
print(len(test_predict))



8447


In [19]:
df=pd.DataFrame({"pssid":test_data_sdss_id,"mass":np.log(test_predict)})
df
df[["pssid","mass"]].to_csv(os.path.join(output_path,"subMath_fix_"+version+"_"+str(image_size)+".csv"), index=False)

In [20]:
df.head()

,mass,pssid
0,10.145727,1237662302977851635
1,9.696742,1237652899687104664
2,9.168529,1237649954407907520
3,10.208810,1237661055281856813
4,11.684472,1237651272957689913
